In [1]:
import os
import vllm
import json
from easysteer.hidden_states import get_all_hidden_states
from vllm import LLM

from easysteer.steer import extract_pca_control_vector,StatisticalControlVector


os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"



model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"

# vector_path = "vectors/thinking_switch_pca_MATH-500.gguf" #MATH-500


num_question = 20

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-12 21:34:40 [__init__.py:244] Automatically detected platform cuda.


In [2]:
file_path = "../../temp/test.jsonl" #MATH-500
# file_path = "../../temp/test_gsm8k.jsonl" #GSM8K

vector_path = "MATH500.gguf"

In [3]:
problem_list = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        line_count = 0
        for line in f:
            if line_count >= num_question:
                break
            stripped_line = line.strip()
            if not stripped_line:
                continue
                
            try:
                data = json.loads(stripped_line)
                if "problem" in data:
                    problem_list.append(data["problem"])
                    line_count += 1
                elif "question" in data:
                    problem_list.append(data["question"])
                    line_count += 1
            except json.JSONDecodeError:
                print(f"skip: {line[:50]}...")
                continue

except Exception as e:
    print(f"{str(e)}")
# problem_list = ["Find the roots of $(x - 3)^3 + (x -7)^3 = (2x - 10)^3.$","A regular hexagon can be divided into six equilateral triangles. If the perimeter of one of the triangles is 21 inches, what is the perimeter, in inches, of the regular hexagon?",""]

In [4]:
slow_thinking = [
    "Walk me through your reasoning process step by step."
]

fast_thinking = [
    "Direct answer only."
]

len_slow=len(slow_thinking)
len_fast=len(fast_thinking)

In [5]:
texts = []

for prob in problem_list:
    for req in slow_thinking:
        texts.append(f"""<|im_start|>user
{prob}{req}<|im_end|>
<|im_start|>assistant
""")

for prob in problem_list:
    for req in fast_thinking:
        texts.append(f"""<|im_start|>user
{prob}{req}<|im_end|>
<|im_start|>assistant
""")
print(texts[0])
print(texts[num_question*len_slow-1])
print(texts[num_question*len_slow])
with open("111.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text + "\n")  # 每个元素后加换行符


<|im_start|>user
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$Walk me through your reasoning process step by step.<|im_end|>
<|im_start|>assistant

<|im_start|>user
If $f(x) = \frac{3x-2}{x-2}$, what is the value of $f(-2) +f(-1)+f(0)$? Express your answer as a common fraction.Walk me through your reasoning process step by step.<|im_end|>
<|im_start|>assistant

<|im_start|>user
Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$Direct answer only.<|im_end|>
<|im_start|>assistant



In [6]:
llm = LLM(model=model_path,task="reward",tensor_parallel_size=1)

all_hidden_states, outputs = get_all_hidden_states(llm, texts)

INFO 07-12 21:34:49 [config.py:1472] Using max model len 32768


2025-07-12 21:34:49,720	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-12 21:34:49 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-12 21:34:49 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.12it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.12it/s]


INFO 07-12 21:34:52 [default_loader.py:272] Loading weights took 0.99 seconds


INFO 07-12 21:34:52 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.140071 seconds


Processed prompts: 100%|██████████| 6/6 [00:00<00:00, 16.91it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [7]:

control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(num_question*len_slow)), 
    negative_indices=list(range(num_question*len_slow,num_question*(len_slow+len_fast))),
    model_type="qwen2.5",
    token_pos=-1,
    normalize=True
)

Computing PCA directions: 100%|██████████| 28/28 [00:00<00:00, 1217.81it/s]


In [8]:
control_vector.export_gguf(vector_path)
control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([ 0.00726394, -0.01336981,  0.00655949, ..., -0.02067229,
        -0.01796948,  0.0129575 ], shape=(1536,), dtype=float32), 1: memmap([ 0.01806596,  0.03636302, -0.00653954, ...,  0.01222791,
         0.02215214, -0.02101433], shape=(1536,), dtype=float32), 2: memmap([ 0.0071452 , -0.01424347, -0.01344561, ..., -0.01061083,
        -0.02976646, -0.0511036 ], shape=(1536,), dtype=float32), 3: memmap([-0.01978249,  0.03541886,  0.01452238, ...,  0.00620668,
         0.02184932, -0.00258583], shape=(1536,), dtype=float32), 4: memmap([-0.02166593,  0.00685351,  0.0129449 , ..., -0.03289864,
         0.02539678, -0.01135035], shape=(1536,), dtype=float32), 5: memmap([-0.00637604, -0.03049226, -0.0374089 , ..., -0.0454876 ,
        -0.00044487,  0.00012361], shape=(1536,), dtype=float32), 6: memmap([ 0.01123103,  0.0168045 ,  0.04298282, ...,  0.0172068 ,
        -0.0007676 ,  0.03093815], shape=(1536,), dtype